<a href="https://colab.research.google.com/github/LiamDuero03/DS-Society-Project/blob/main/Full-Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Data Sourcing
Goal: Load data from a persistent source so the code never breaks for teammates.

In [ ]:
# --- SECTION 1: DATA SOURCING ---
import pandas as pd

# Pro-tip: Host your CSV on GitHub and use the 'Raw' link
DATA_URL = "https://raw.githubusercontent.com/your-username/your-repo/main/data.csv"

def load_data():
    df = pd.read_csv(DATA_URL)
    print(f"Dataset Loaded: {df.shape[0]} rows and {df.shape[1]} columns")
    return df

df = load_data()

2. Preliminary EDA & Visualization
Goal: Understand the "shape" and "health" of the data.

In [ ]:
# --- SECTION 2: PRELIMINARY EDA ---
import seaborn as sns
import matplotlib.pyplot as plt

# Check for missing values and data types
print(df.info())

# Visualizing the target variable distribution
sns.histplot(df['target_column'], kde=True)
plt.title("Distribution of Target Variable")
plt.show()

3. Data Wrangling
Goal: Clean the data for the machine. This is where your team will spend most of their time.

In [ ]:
# --- SECTION 3: DATA WRANGLING ---
# 1. Handling Missing Values
df['column'] = df['column'].fillna(df['column'].median())

# 2. Encoding Categorical Variables
df = pd.get_dummies(df, columns=['category_column'], drop_first=True)

# 3. Feature Scaling (Week 4 topic)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# df[['num_cols']] = scaler.fit_transform(df[['num_cols']])

4. Advanced Visualization
Goal: Storytelling. Move beyond simple bars to multi-dimensional charts.

In [ ]:
# --- SECTION 4: ADVANCED VISUALS ---
import plotly.express as px

# Create an interactive 3D scatter or a Correlation Heatmap
corr = df.corr()
fig = px.imshow(corr, text_auto=True, title="Feature Correlation Heatmap")
fig.show()

5. Machine Learning (ML)
Goal: The predictive engine.

In [ ]:
# --- SECTION 5: MACHINE LEARNING ---
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # or Classifier

X = df.drop('target_column', axis=1)
y = df['target_column']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(X_train, y_train)
print(f"Model Score: {model.score(X_test, y_test)}")